In [1]:
"""
导入相关库进入命名空间.
"""

import numpy as np
import pandas as pd
import statsmodels.api as sm
import linearmodels.panel as panel

<br>

---

<br>

In [2]:
"""
读取pwt数据.
数据来源: 
<https://www.rug.nl/ggdc/docs/pwt100.xlsx>
"""

# data = pd.io.stata.read_stata("pwt100.dta")
excel_file = pd.ExcelFile("pwt100.xlsx")  
sheet_names = excel_file.sheet_names[1:]   

for sh in sheet_names:
    print("Converting \033[32m{:s}\033[0m into DataFrame.".format(sh))
    globals()[sh.lower()] = excel_file.parse(sh)

Converting Legend into DataFrame.
Converting Data into DataFrame.


In [3]:
# data.head()
data.tail()

,countrycode,country,currency_unit,year,rgdpe,rgdpo,pop,emp,avh,hc,...,csh_x,csh_m,csh_r,pl_c,pl_i,pl_g,pl_x,pl_m,pl_n,pl_k
12805,ZWE,Zimbabwe,US Dollar,2015,40141.617188,39798.644531,13.814629,6.393752,NaN,2.584653,...,0.140172,-0.287693,-0.051930,0.479228,0.651287,0.541446,0.616689,0.533235,0.422764,1.534175
12806,ZWE,Zimbabwe,US Dollar,2016,41875.203125,40963.191406,14.030331,6.504374,NaN,2.616257,...,0.131920,-0.251232,-0.016258,0.470640,0.651027,0.539631,0.619789,0.519718,0.416510,1.492129
12807,ZWE,Zimbabwe,US Dollar,2017,44672.175781,44316.742188,14.236595,6.611773,NaN,2.648248,...,0.126722,-0.202827,-0.039897,0.473560,0.639560,0.519956,0.619739,0.552042,0.415592,1.515128
12808,ZWE,Zimbabwe,US Dollar,2018,44325.109375,43420.898438,14.438802,6.714952,NaN,2.680630,...,0.144485,-0.263658,-0.020791,0.543757,0.655473,0.529867,0.641361,0.561526,0.425143,1.590753
12809,ZWE,Zimbabwe,US Dollar,2019,42296.062500,40826.570312,14.645468,6.831017,NaN,2.713408,...,0.213562,-0.270959,-0.089798,0.494755,0.652439,0.500927,0.487763,0.430082,0.420675,1.384638


In [4]:
data.shape

(12810, 52)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12810 entries, 0 to 12809
Data columns (total 52 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   countrycode    12810 non-null  object 
 1   country        12810 non-null  object 
 2   currency_unit  12810 non-null  object 
 3   year           12810 non-null  int64  
 4   rgdpe          10399 non-null  float64
 5   rgdpo          10399 non-null  float64
 6   pop            10399 non-null  float64
 7   emp            9529 non-null   float64
 8   avh            3492 non-null   float64
 9   hc             8637 non-null   float64
 10  ccon           10399 non-null  float64
 11  cda            10399 non-null  float64
 12  cgdpe          10399 non-null  float64
 13  cgdpo          10395 non-null  float64
 14  cn             10314 non-null  float64
 15  ck             7095 non-null   float64
 16  ctfp           6412 non-null   float64
 17  cwtfp          6412 non-null   float64
 18  rgdpna

In [6]:
data.isnull()

,countrycode,country,currency_unit,year,rgdpe,rgdpo,pop,emp,avh,hc,...,csh_x,csh_m,csh_r,pl_c,pl_i,pl_g,pl_x,pl_m,pl_n,pl_k
0,False,False,False,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,False,False,False,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,False,False,False,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,False,False,False,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,False,False,False,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12805,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
12806,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
12807,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
12808,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
"""
查看data Sheet中各列代码详细解释. 
"""

# legend.set_index(legend.columns[0])
legend.index = legend.iloc[:, 0] 
legend.drop(legend.columns[0], axis=1, inplace=True)

legend.dropna(how="all", inplace=True)  
legend.dropna(axis=1, inplace=True)

for idx, val in legend.itertuples():  
    print("<\033[34m{:s}\033[0m>\n{}\n".format(idx, val))

<countrycode>
3-letter ISO country code

<country>
Country name

<currency_unit>
Currency unit

<year>
Year

<rgdpe>
Expenditure-side real GDP at chained PPPs (in mil. 2017US$)

<rgdpo>
Output-side real GDP at chained PPPs (in mil. 2017US$)

<pop>
Population (in millions)

<emp>
Number of persons engaged (in millions)

<avh>
Average annual hours worked by persons engaged

<hc>
Human capital index, based on years of schooling and returns to education; see Human capital in PWT9.

<ccon>
Real consumption of households and government, at current PPPs (in mil. 2017US$)

<cda>
Real domestic absorption, (real consumption plus investment), at current PPPs (in mil. 2017US$)

<cgdpe>
Expenditure-side real GDP at current PPPs (in mil. 2017US$)

<cgdpo>
Output-side real GDP at current PPPs (in mil. 2017US$)

<cn>
Capital stock at current PPPs (in mil. 2017US$)

<ck>
Capital services levels at current PPPs (USA=1)

<ctfp>
TFP level at current PPPs (USA=1)

<cwtfp>
Welfare-relevant TFP levels at cur

<br>

---

<br>

In [8]:
"""
读取民主指标数据. 
选取L_A1指标, 详见:
<https://www.idea.int/data-tools/tools/global-state-democracy-indices>
数据来源: 
<https://www.idea.int/gsod-indices/sites/default/files/gsodi_pv_4.csv>
"""

democracy_indecies = pd.read_csv("gsodi_pv_4.csv", low_memory=False)

<br>

---

<br>

In [9]:
"""
根据条件清洗数据, 并建模.
"""

state_array = np.intersect1d(
        data["country"].unique(), 
        democracy_indecies["ID_country_name"].unique()
) 
cols = ["country", "year", "cda", "pop", "csh_i", 
        "csh_g", "csh_x", "csh_m", "ctfp"] 
pwt = data[cols].copy()  
pwt.dropna(inplace=True) 

post_data = pd.DataFrame()
for var in state_array:  
    state = pwt.loc[pwt["country"] == var].copy()  
    demo = democracy_indecies.loc[democracy_indecies["ID_country_name"] == var]
    
    if (len(state) >= 46) and (len(demo) == 45):  
        cda = state["cda"].values
        growth = ((cda[1:]-cda[:-1]) / cda[:-1])[-45:] 
        tmp = state[-45:].copy()
        tmp.insert(0, "growth", growth)  
        tmp.insert(0, "democracy", demo["C_SD13"].values) 
        post_data = pd.concat([post_data, tmp])
        
post_data.insert(0, "csh_xm", post_data["csh_x"] + post_data["csh_m"])
post_data.set_index(["country", "year"], inplace=True)

g = post_data["growth"]  
cols = ["pop", "csh_i", "csh_g", "csh_xm", "democracy"]
X = sm.add_constant(post_data[cols]) 

In [10]:
X

const        pop     csh_i     csh_g    csh_xm  democracy
country  year                                                           
Angola   1975    1.0   7.024000  0.409320  0.221240  0.095105   0.072882
         1976    1.0   7.279509  0.396777  0.215925  0.043802   0.109767
         1977    1.0   7.533735  0.387024  0.212701 -0.011606   0.109767
         1978    1.0   7.790707  0.401575  0.224828  0.058939   0.109767
         1979    1.0   8.058067  0.385584  0.223121  0.049473   0.109767
...              ...        ...       ...       ...       ...        ...
Zimbabwe 2015    1.0  13.814629  0.077963  0.176403 -0.147522   0.517929
         2016    1.0  14.030331  0.076169  0.168887 -0.119313   0.513931
         2017    1.0  14.236595  0.075448  0.207101 -0.076105   0.537214
         2018    1.0  14.438802  0.079576  0.269799 -0.119173   0.526900
         2019    1.0  14.645468  0.077089  0.227109 -0.057398   0.526900

[3690 rows x 6 columns]

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3690 entries, ('Angola', 1975) to ('Zimbabwe', 2019)
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   const      3690 non-null   float64
 1   pop        3690 non-null   float64
 2   csh_i      3690 non-null   float64
 3   csh_g      3690 non-null   float64
 4   csh_xm     3690 non-null   float64
 5   democracy  3690 non-null   float64
dtypes: float64(6)
memory usage: 184.4+ KB


In [12]:
model = panel.PanelOLS(g, X, entity_effects=True, time_effects=True) 
result = model.fit(cov_type="unadjusted")

print(result.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 growth   R-squared:                        0.0332
Estimator:                   PanelOLS   R-squared (Between):             -1.0602
No. Observations:                3690   R-squared (Within):               0.0373
Date:                Thu, Jun 03 2021   R-squared (Overall):             -0.0097
Time:                        14:37:14   Log-likelihood                    4061.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      24.414
Entities:                          82   P-value                           0.0000
Avg Obs:                       45.000   Distribution:                  F(5,3559)
Min Obs:                       45.000                                           
Max Obs:                       45.000   F-statistic (robust):             24.414
                            

In [13]:
?panel.PanelOLS

Init signature:
panel.PanelOLS(
    dependent: Union[linearmodels.panel.data.PanelData, numpy.ndarray, pandas.core.frame.DataFrame, pandas.core.series.Series],
    exog: Union[linearmodels.panel.data.PanelData, numpy.ndarray, pandas.core.frame.DataFrame, pandas.core.series.Series],
    *,
    weights: Union[linearmodels.panel.data.PanelData, numpy.ndarray, pandas.core.frame.DataFrame, pandas.core.series.Series, NoneType] = None,
    entity_effects: bool = False,
    time_effects: bool = False,
    other_effects: Union[linearmodels.panel.data.PanelData, numpy.ndarray, pandas.core.frame.DataFrame, pandas.core.series.Series, NoneType] = None,
    singletons: bool = True,
    drop_absorbed: bool = False,
    check_rank: bool = True,
) -> None
Docstring:     
One- and two-way fixed effects estimator for panel data

Parameters
----------
dependent : array_like
    Dependent (left-hand-side) variable (time by entity).
exog : array_like
    Exogenous or right-hand-side variables (variable by t

<br>

---

<br>

In [14]:
"""
预测变量中添加tfp. Penn world table中的ctfp数据为当前购买力平价下的tfp.
数据来源: penn world table.
"""

cols = ["pop", "csh_i", "csh_g", "csh_xm", "democracy", "ctfp"]
Y = sm.add_constant(post_data[cols])
model = panel.PanelOLS(g, Y, entity_effects=True, time_effects=True)
result = model.fit(cov_type="unadjusted")

print(result.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 growth   R-squared:                        0.0648
Estimator:                   PanelOLS   R-squared (Between):             -4.4653
No. Observations:                3690   R-squared (Within):               0.0575
Date:                Thu, Jun 03 2021   R-squared (Overall):             -0.1362
Time:                        14:37:15   Log-likelihood                    4123.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      41.084
Entities:                          82   P-value                           0.0000
Avg Obs:                       45.000   Distribution:                  F(6,3558)
Min Obs:                       45.000                                           
Max Obs:                       45.000   F-statistic (robust):             41.084
                            